![image.png](attachment:image.png)

In [ ]:
from tensorflow.keras import Sequential, Model, layers

enc_input = layers.Input(shape=(4, 171))
enc_output, state_h, state_c = layers.LSTM(128, return_sequences=True, return_state=True)(enc_input)

dec_input = layers.Input(shape=(3, 171)) # 한글 단어 2자와 Start 토큰 1개, 총 3개가 디코더의 입력으로 들어간다.
dec_output = layers.LSTM(128, return_sequences=True)(dec_input, initial_state=[state_h, state_c])

output = layers.Dense(171, activation='softmax')(dec_output) # 작명 이슈, 사실 이게 디코더 아웃풋

model = Model(inputs=[enc_input, dec_input], outputs=output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 4, 171)]             0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 3, 171)]             0         []                            
                                                                                                  
 lstm (LSTM)                 [(None, 4, 128),             153600    ['input_1[0][0]']             
                              (None, 128),                                                        
                              (None, 128)]                                                        
                                                                                              

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [ ]:
# 데이터 불러오기, (영어, 한글, 단어 사전)
import numpy as np
import pandas as pd

# arr1 = ['S', 'E', 'P', 'a', 'b', ...]
# 이렇게 코딩하면 너무 번거로워서 아래와 같이 사용
arr1 = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz'] # 리스트 컨프렌션?
print(arr1)

arr2 = pd.read_csv('korean.csv', header=None)
# print(arr2[0].values.tolist()) # 베열은 ,(콤마가 안 보임), list는 (,)콤마가 보임
num_to_char = arr1 + arr2[0].values.tolist()
print(len(num_to_char))

['S', 'E', 'P', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
171


In [ ]:
char_to_num = {char: i for i, char in enumerate(num_to_char)}
# print(char_to_num)

In [ ]:
raw = pd.read_csv('translate.csv', header=None)
raw.head()
eng_kor = raw.values.tolist()
print(len(eng_kor))

110


In [ ]:
temp_eng = 'love'
temp_eng_n = [char_to_num[c] for c in temp_eng]
print(temp_eng_n)

[14, 17, 24, 7]


In [ ]:
temp_kor = '사랑'
temp_kor_n = [char_to_num[c] for c in temp_kor]
print(temp_kor_n)

[96, 66]


In [ ]:
np.eye(5)[[4, 3, 1]]
# 단위행렬
# 인덱싱 배열을 이용한 인덱싱 (numpy 부분 복습하면 알 수 있음)

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.]])

In [ ]:
np.eye(171)[[14, 17, 24, 7]].shape

(4, 171)

In [ ]:
def encode(eng_kor): # eng_kor = [['love', '사랑'], ['...', '...']]
    enc_in = []
    dec_in = []
    rnn_out = []

    for seq in eng_kor:
        eng = [char_to_num[c] for c in seq[0]]
        enc_in.append(np.eye(171)[eng]) # eng 리스트로 인덱싱

        kor = [char_to_num[c] for c in 'S'+seq[1]]
        dec_in.append(np.eye(171)[kor]) # kor 리스트로 인덱싱

        target = [char_to_num[c] for c in seq[1]+'E']
        rnn_out.append(target) # loss(손실함수)에서 sparse를 지정했기 때문에 원-핫 인코딩을 하지 않고 생략

    enc_in = np.array(enc_in)
    dec_in = np.array(dec_in)
    rnn_out = np.expand_dims(np.array(rnn_out), axis=2)

    return enc_in, dec_in, rnn_out

In [ ]:
sample = [['word', '단어']]
encode(sample)[2]

array([[[ 61],
        [114],
        [  1]]])

In [ ]:
X_enc, X_dec, y_rnn = encode(eng_kor)

In [ ]:
model.fit([X_enc, X_dec], y_rnn, epochs=500)

Epoch 1/500
4/4 [==============================] - 4s 13ms/step - loss: 5.1321
Epoch 2/500
4/4 [==============================] - 0s 13ms/step - loss: 5.0925
Epoch 3/500
4/4 [==============================] - 0s 14ms/step - loss: 5.0414
Epoch 4/500
4/4 [==============================] - 0s 14ms/step - loss: 4.9569
Epoch 5/500
4/4 [==============================] - 0s 11ms/step - loss: 4.7955
Epoch 6/500
4/4 [==============================] - 0s 11ms/step - loss: 4.4516
Epoch 7/500
4/4 [==============================] - 0s 11ms/step - loss: 3.8091
Epoch 8/500
4/4 [==============================] - 0s 13ms/step - loss: 3.5172
Epoch 9/500
4/4 [==============================] - 0s 14ms/step - loss: 3.5976
Epoch 10/500
4/4 [==============================] - 0s 12ms/step - loss: 3.4246
Epoch 11/500
4/4 [==============================] - 0s 13ms/step - loss: 3.3719
Epoch 12/500
4/4 [==============================] - 0s 13ms/step - loss: 3.3779
Epoch 13/500
4/4 [==============================]

4/4 [==============================] - 0s 12ms/step - loss: 0.4093
Epoch 104/500
4/4 [==============================] - 0s 9ms/step - loss: 0.3935
Epoch 105/500
4/4 [==============================] - 0s 9ms/step - loss: 0.3766
Epoch 106/500
4/4 [==============================] - 0s 10ms/step - loss: 0.3627
Epoch 107/500
4/4 [==============================] - 0s 9ms/step - loss: 0.3482
Epoch 108/500
4/4 [==============================] - 0s 10ms/step - loss: 0.3364
Epoch 109/500
4/4 [==============================] - 0s 9ms/step - loss: 0.3241
Epoch 110/500
4/4 [==============================] - 0s 10ms/step - loss: 0.3114
Epoch 111/500
4/4 [==============================] - 0s 9ms/step - loss: 0.3010
Epoch 112/500
4/4 [==============================] - 0s 10ms/step - loss: 0.2899
Epoch 113/500
4/4 [==============================] - 0s 10ms/step - loss: 0.2795
Epoch 114/500
4/4 [==============================] - 0s 9ms/step - loss: 0.2702
Epoch 115/500
4/4 [=============================

4/4 [==============================] - 0s 12ms/step - loss: 0.0379
Epoch 205/500
4/4 [==============================] - 0s 13ms/step - loss: 0.0374
Epoch 206/500
4/4 [==============================] - 0s 13ms/step - loss: 0.0369
Epoch 207/500
4/4 [==============================] - 0s 10ms/step - loss: 0.0364
Epoch 208/500
4/4 [==============================] - 0s 11ms/step - loss: 0.0359
Epoch 209/500
4/4 [==============================] - 0s 10ms/step - loss: 0.0354
Epoch 210/500
4/4 [==============================] - 0s 10ms/step - loss: 0.0350
Epoch 211/500
4/4 [==============================] - 0s 10ms/step - loss: 0.0347
Epoch 212/500
4/4 [==============================] - 0s 10ms/step - loss: 0.0341
Epoch 213/500
4/4 [==============================] - 0s 9ms/step - loss: 0.0337
Epoch 214/500
4/4 [==============================] - 0s 9ms/step - loss: 0.0334
Epoch 215/500
4/4 [==============================] - 0s 9ms/step - loss: 0.0329
Epoch 216/500
4/4 [==========================

4/4 [==============================] - 0s 10ms/step - loss: 0.0134
Epoch 307/500
4/4 [==============================] - 0s 9ms/step - loss: 0.0133
Epoch 308/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0132
Epoch 309/500
4/4 [==============================] - 0s 9ms/step - loss: 0.0131
Epoch 310/500
4/4 [==============================] - 0s 9ms/step - loss: 0.0130
Epoch 311/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0129
Epoch 312/500
4/4 [==============================] - 0s 10ms/step - loss: 0.0128
Epoch 313/500
4/4 [==============================] - 0s 10ms/step - loss: 0.0127
Epoch 314/500
4/4 [==============================] - 0s 9ms/step - loss: 0.0126
Epoch 315/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0125
Epoch 316/500
4/4 [==============================] - 0s 9ms/step - loss: 0.0124
Epoch 317/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0123
Epoch 318/500
4/4 [==============================] 

4/4 [==============================] - 0s 10ms/step - loss: 0.0068
Epoch 409/500
4/4 [==============================] - 0s 9ms/step - loss: 0.0068
Epoch 410/500
4/4 [==============================] - 0s 10ms/step - loss: 0.0068
Epoch 411/500
4/4 [==============================] - 0s 10ms/step - loss: 0.0067
Epoch 412/500
4/4 [==============================] - 0s 10ms/step - loss: 0.0067
Epoch 413/500
4/4 [==============================] - 0s 13ms/step - loss: 0.0066
Epoch 414/500
4/4 [==============================] - 0s 10ms/step - loss: 0.0066
Epoch 415/500
4/4 [==============================] - 0s 9ms/step - loss: 0.0066
Epoch 416/500
4/4 [==============================] - 0s 9ms/step - loss: 0.0065
Epoch 417/500
4/4 [==============================] - 0s 9ms/step - loss: 0.0065
Epoch 418/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0065
Epoch 419/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0064
Epoch 420/500
4/4 [=============================

In [ ]:
input_data = [['love', 'PP']]
in_, out_, _ = encode(input_data)

pred = model.predict([in_, out_])
# print(pred)

word = np.argmax(pred[0], axis=1)
print(word)

print(num_to_char[word[0]], num_to_char[word[1]])

1/1 [==============================] - 0s 22ms/step
[96 66  1]
사 랑
